<a href="https://colab.research.google.com/github/sahupra1357/Pawpularity/blob/main/PetFinderCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q kaggle==1.5.8

     |████████████████████████████████| 59 kB 2.9 MB/s 


In [2]:
!ls

sample_data


In [3]:
!mkdir -p ~/.kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pradeepsahu1357","key":"b78a1773d2ce934ad103512a70ef714f"}'}

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!ls

kaggle.json  sample_data


In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c petfinder-pawpularity-score

 99% 977M/983M [00:18<00:00, 75.4MB/s]
100% 983M/983M [00:18<00:00, 54.3MB/s]


In [9]:
!ls

kaggle.json  petfinder-pawpularity-score.zip  sample_data


In [ ]:
!unzip petfinder-pawpularity-score.zip

In [11]:
!ls

kaggle.json			 sample_data		test	  train
petfinder-pawpularity-score.zip  sample_submission.csv	test.csv  train.csv


In [12]:
!ls train |wc -l

9912


In [13]:
!ls test |wc -l

8


In [14]:
import pandas as pd

df_training = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_training["Id"] = df_training["Id"].apply(lambda x :  x + ".jpg")
df_test["Id"] = df_test["Id"].apply(lambda x : x + ".jpg")

In [15]:
df_training.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,filename
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,0007de18844b0dbbb5e1f607da0606e0.jpg
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,0009c66b9439883ba2750fb825e1d7db.jpg
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,0013fd999caf9a3efe1352ca1b0d937e.jpg
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,0018df346ac9c1d8413cfcc888ca8246.jpg
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,001dc955e10590d3ca4673f034feeef2.jpg


In [16]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_training) * TRAIN_PCT)

df_train = df_training[0:TRAIN_CUT]
df_validate = df_training[TRAIN_CUT:]

print(f"Training size: {len(df_train)}")
print(f"Validate size: {len(df_validate)}")

Training size: 8920
Validate size: 992


In [17]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import os

IMAGES_DIR = os.path.abspath("train"+os.sep)

In [18]:
training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_DIR,
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        batch_size=32,
        class_mode='other')

Found 8920 validated image filenames.


In [19]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate,
        directory=IMAGES_DIR,
        x_col="Id",
        y_col="Pawpularity",
        target_size=(256, 256),
        class_mode='other')

Found 992 validated image filenames.


In [20]:
from tensorflow.keras.callbacks import EarlyStopping

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 512)               1

In [21]:
model.compile(loss = 'mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
history = model.fit(train_generator,  
  verbose = 1, 
  validation_data=val_generator, callbacks=[monitor], epochs=25)

Epoch 1/25
279/279 [==============================] - 168s 489ms/step - loss: 547.9589 - val_loss: 422.0881
Epoch 2/25
279/279 [==============================] - 137s 489ms/step - loss: 444.1037 - val_loss: 425.1378
Epoch 3/25
279/279 [==============================] - 135s 482ms/step - loss: 439.0573 - val_loss: 417.9485
Epoch 4/25
279/279 [==============================] - 134s 481ms/step - loss: 437.0948 - val_loss: 413.5647
Epoch 5/25
279/279 [==============================] - 135s 484ms/step - loss: 433.8019 - val_loss: 410.6241
Epoch 6/25
279/279 [==============================] - 135s 483ms/step - loss: 425.3116 - val_loss: 408.3762
Epoch 7/25
279/279 [==============================] - 135s 485ms/step - loss: 420.0025 - val_loss: 410.3288
Epoch 8/25
279/279 [==============================] - 135s 482ms/step - loss: 413.3625 - val_loss: 411.2820
Epoch 9/25
279/279 [==============================] - 134s 481ms/step - loss: 403.8938 - val_loss: 435.3368
Epoch 10/25
279/279 [=======

In [23]:
IMAGES_DIR_TEST = os.path.abspath("test"+os.sep)

In [24]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=IMAGES_DIR_TEST,
        x_col="Id",
        batch_size=1,
        shuffle=False,
        target_size=(256, 256),
        class_mode=None)

Found 8 validated image filenames.


In [25]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))

In [26]:
df_submit = pd.DataFrame({'Id':df_test['Id'],'Pawpularity':pred.flatten()})

In [27]:
df_submit.to_csv("submit.csv",index=False)

In [28]:
from google.colab import files
files.download("submit.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>